<a href="https://colab.research.google.com/github/TW-ZJLin/G-CodeInterpreter/blob/main/G_CODE_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import numpy as np
from google.colab import drive
import plotly.express as px
import plotly.graph_objs as go

In [12]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

import plotly.io as pio

pio.write_html(fig, file='figure.html', auto_open=True)

## Link to Google Drive

In [ ]:
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/DATA

def ReadFile(file_path):
  f = open('/content/drive/My Drive/Colab Notebooks/NC Code/' + file_path, 'r')
  NC_file = f.read()
  f.close()
  return NC_file

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/DATA


## Function Definition

In [ ]:
#-------------------------------------------------------------------------------
# Introduction: NC code interpreter
# Input: (list) each line of the NC code
# Output: (array of numpy) X, Y, Z, R, F for each columns
#-------------------------------------------------------------------------------
def NCCodeInterpreter(NC_list):
  # Initialization
  #-------------------------------------------------------------------------------
  DATA = [[0,0,0,0,0]]

  IsIncremental = 0
  IsCycle = 0

  filter_set = { 'X','Y','Z','R','F' }
  G_set = { 'G00','G01','G02','G03','G90','G91' }
  digital_set = { '0','1','2','3','4','5','6','7','8','9','0','-','+','.',' ' }

  # Loop for each line of the NC code
  #-------------------------------------------------------------------------------
  for i in range(len(NC_list)):
    if not NC_list[i]: continue

    if 'G90' in NC_list[i]:
      IsIncremental = 0 # Absolute Coordinates
    elif 'G91' in NC_list[i]:
      IsIncremental = 1 # Incremental Coordinates

    if 'G01' in NC_list[i] or 'G00' in NC_list[i]:
      IsCycle = 0 # Line
    elif 'G03' in NC_list[i]:
      IsCycle = 1 # CCW
    elif 'G02' in NC_list[i]:
      IsCycle = -1 # CW
    
    # First word must be X, Y, Z, R, F or G
    if NC_list[i][0] in filter_set.union({'G'}):
      # Update Data Initialization
      #-------------------------------------------------------------------------------
      data_update = [list(DATA[-1])]
      data_update[0][3] = 0.

      # Find X, Y, Z, R and F
      #-------------------------------------------------------------------------------
      for init_text in range(len(NC_list[i])):
        end_text = 1
        if NC_list[i][init_text] in filter_set:
          while( init_text+end_text < len(NC_list[i])):
            if NC_list[i][init_text+end_text] in digital_set:
              end_text += 1
            else:
              break
          
          # Save Data
          #-------------------------------------------------------------------------------
          if NC_list[i][init_text] == 'X':
            data_update[0][0] = IsIncremental*data_update[0][0] + float( NC_list[i][init_text+1:init_text+end_text] )
          elif NC_list[i][init_text] == 'Y':
            data_update[0][1] = IsIncremental*data_update[0][1] + float( NC_list[i][init_text+1:init_text+end_text] )
          elif NC_list[i][init_text] == 'Z':
            data_update[0][2] = IsIncremental*data_update[0][2] + float( NC_list[i][init_text+1:init_text+end_text] )
          elif NC_list[i][init_text] == 'R':
            data_update[0][3] = IsCycle*float( NC_list[i][init_text+1:init_text+end_text] )
          elif NC_list[i][init_text] == 'F':
            data_update[0][4] = float( NC_list[i][init_text+1:init_text+end_text] )

      # Updata Data
      #-------------------------------------------------------------------------------
      DATA += data_update

  # Continuous Data Deduplication
  #-------------------------------------------------------------------------------
  DATA = np.array(DATA)
  delete_idx = []
  for i in range(DATA.shape[0]-1):
    if np.array_equal( DATA[i], DATA[i+1] ):
      delete_idx += [i]
  DATA = np.delete(DATA,delete_idx,axis=0)

  # Reorganize
  #-------------------------------------------------------------------------------
  DATA = np.hstack([ DATA[:,:3], np.roll(DATA[:,3:4],-1), DATA[:,4:] ])

  return DATA

In [ ]:
#-------------------------------------------------------------------------------
# Introduction: Calculation for centre of circle and points of arc
# Known: two points A(x1,y1) and B(x2,y2) with radius R
# Input: (float) A(x1,y1) = initial position; B(x2,y2) = end position;
#          r = radius; resolution = number of points for each line
# Output: (list) cal_x, cal_y collect point positions on the arcs
#-------------------------------------------------------------------------------
def CycleCalculation(x1,x2,y1,y2,r,resolution):
  cal_x = []
  cal_y = []
  
  # Vector and angle
  #-------------------------------------------------------------------------------
  vec = np.array( [ x2 - x1 , y2 - y1 ], dtype=float)
  theta = np.arctan2( vec[1], vec[0] )

  # MO = The vertical distance from the center of the circle to the line AB
  #-------------------------------------------------------------------------------
  if (vec[0]**2+vec[1]**2)/4 > r**2:
    MO = 0.
    print("radius error!")
  else:
    MO = r/abs(r)*np.sqrt(r**2-(vec[0]**2+vec[1]**2)/4)

  # centre = center of the circle
  #-------------------------------------------------------------------------------
  centre = np.array([ (x2+x1)/2 - MO*np.sin(theta) , (y2+y1)/2 + MO*np.cos(theta) ], dtype=float)
  
  # Ctheta = starting phase angle
  #-------------------------------------------------------------------------------
  Ctheta = np.arctan2( y1-centre[1], x1-centre[0] )

  # Set points
  #-------------------------------------------------------------------------------
  for j in range(resolution):
    the = Ctheta + j*( r/abs(r)*2*np.arcsin( np.sqrt(vec[0]**2+vec[1]**2) / (2*abs(r)) ) )/resolution
    x = centre[0] + abs(r)*np.cos(the)
    y = centre[1] + abs(r)*np.sin(the)
    cal_x = cal_x + [x]
    cal_y = cal_y + [y]
  
  return cal_x, cal_y

In [ ]:
#-------------------------------------------------------------------------------
# Introduction: set points for plotting
# Input: DATA(array of numpy) X, Y, Z, R, F for each columns
#     axes(int) number of axes
# Output: nominal_x, nominal_y for 2 axes
#     nominal_x, nominal_y, nominal_z for 3 axes
#-------------------------------------------------------------------------------
def Nominal(DATA,axes=2,resolution=10):
  if axes == 2:
    nominal_x = []
    nominal_y = []
    for i in range(DATA.shape[0]-1):
      if DATA[i,2] == 0: # line
        nominal_x = nominal_x + [ DATA[i,0] ]
        nominal_y = nominal_y + [ DATA[i,1] ]
      else: # circle
        cal_x, cal_y = CycleCalculation( DATA[i,0], DATA[i+1,0], DATA[i,1], DATA[i+1,1], DATA[i,2], resolution )
        nominal_x += cal_x
        nominal_y += cal_y
    
    # end point
    nominal_x = nominal_x + [ 0 ]
    nominal_y = nominal_y + [ 0 ]
    nominal_x = np.array(nominal_x)
    nominal_y = np.array(nominal_y)
    return nominal_x, nominal_y
  
  elif axes == 3:
    nominal_x = []
    nominal_y = []
    nominal_z = []
    for i in range(DATA.shape[0]-1):
      if DATA[i,3] == 0: #line
        nominal_x = nominal_x + [ DATA[i,0] ]
        nominal_y = nominal_y + [ DATA[i,1] ]
        nominal_z = nominal_z + [ DATA[i,2] ]
      else: #circle
        # circle on XY plane
        if DATA[i,0] != DATA[i+1,0] and DATA[i,1] != DATA[i+1,1] and DATA[i,2] == DATA[i+1,2]:
          cal_x, cal_y = CycleCalculation( DATA[i,0], DATA[i+1,0], DATA[i,1], DATA[i+1,1], DATA[i,3], resolution )
          nominal_x += cal_x
          nominal_y += cal_y
          nominal_z += [ DATA[i-1,2] for _ in range(resolution) ] # zero padding

        # circle on YZ plane
        elif DATA[i,0] == DATA[i+1,0] and DATA[i,1] != DATA[i+1,1] and DATA[i,2] != DATA[i+1,2]:
          cal_x, cal_y = CycleCalculation( DATA[i,1], DATA[i+1,1], DATA[i,2], DATA[i+1,2], DATA[i,3], resolution )
          nominal_x += [ DATA[i-1,0] for _ in range(resolution) ] # zero padding
          nominal_y += cal_x
          nominal_z += cal_y

        # circle on ZX plane
        elif DATA[i,0] != DATA[i+1,0] and DATA[i,1] == DATA[i+1,1] and DATA[i,2] != DATA[i+1,2]:
          cal_x, cal_y = CycleCalculation( DATA[i,2], DATA[i+1,2], DATA[i,0], DATA[i+1,0], DATA[i,3], resolution )
          nominal_x += cal_y
          nominal_y += [ DATA[i-1,1] for _ in range(resolution) ] # zero padding
          nominal_z += cal_x

        # line
        else:
          nominal_x += [ DATA[i,0] ]
          nominal_y += [ DATA[i,1] ]
          nominal_z += [ DATA[i,2] ]

    # end point
    nominal_x += [ 0 ]
    nominal_y += [ 0 ]
    nominal_z += [ 0 ]
    nominal_x = np.array(nominal_x)
    nominal_y = np.array(nominal_y)
    nominal_z = np.array(nominal_z)
    return nominal_x, nominal_y, nominal_z

In [ ]:
# 3D drawing with Plotly
#-------------------------------------------------------------------------------
def Plotly3D(DATA,axes=3,resolution=10):
  # 3Axes
  if DATA.shape[1] == 5:
    nominal_x, nominal_y, nominal_z = Nominal(DATA,axes=3,resolution=10)
    trace1 = px.line_3d( x=nominal_x, y=nominal_y, z=nominal_z )
  # 2Axes
  elif DATA.shape[1] == 4:
    nominal_x, nominal_y = Nominal(DATA,axes=2,resolution=10)
    trace1 = px.line_3d( x=nominal_x, y=nominal_y, z=np.zeros(len(nominal_x)) )
  else:
    print("DATA shape not allow!")
    return 0
    
  fig1 = go.Figure(data=trace1)
  fig1.update_traces(line=dict(color="Black", width=2))

  fig = go.Figure( data = fig1.data )

  # Layout
  #-------------------------------------------------------------------------------
  fig.update_layout(
    width=700,
    height=700,
    autosize=False,
    scene=dict(
      camera=dict(
          eye=dict(
              x = -1.5,
              y = -1.5,
              z = 1.,
          )
      ),
    ),
  )

  return fig

## Implement

### File Name List

In [ ]:
# 'S21CNAB032.txt'
# 'O1224.txt'
# 'XY_KAKINO.txt'
# '110_rough_good.txt'
# '110_finish_good.txt'
# 'O1001(賓士模Part1).nc'
# 'O2001(賓士模Part2).nc'
# 'O9405(Spiral).nc'

### Case 1

In [ ]:
NC_file = ReadFile('110_finish_good.txt')
NC_list = NC_file.upper().split("\n")

DATA = NCCodeInterpreter(NC_list)
Fig = Plotly3D(DATA)
Fig.show()

### Case 2

In [ ]:
NC_file = ReadFile('O1001(賓士模Part1).nc')
NC_list = NC_file.upper().split("\n")

DATA = NCCodeInterpreter(NC_list)
Fig = Plotly3D(DATA[:20000])
Fig.show()

### Case 3

In [ ]:
NC_file = ReadFile('O2001(賓士模Part2).nc')
NC_list = NC_file.upper().split("\n")

DATA = NCCodeInterpreter(NC_list)
Fig = Plotly3D(DATA[:20000])
Fig.show()